In [1]:
import tensorflow as tf
import keras
import numpy as np
print(tf.version.VERSION)
print(keras.__version__)


2.15.0
3.0.2


In [5]:
def get_model():
    global model
    model = keras.models.load_model('mexican_emo_ser/models/model2_cnn2d.h5')
    print('Modelo cargado**')
    
get_model()

c:\Users\AlxMa\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


Modelo cargado**


In [6]:
import librosa

path = r'C:\Users\AlxMa\Desktop\Tesis\MESD\cy34mh68j9-4\cy34mh68j9-4\Mexican Emotional Speech Database (MESD)'
sampling_rate = 16000

def load_audio(audio_path):
    audio,sr = librosa.load(audio_path, sr=sampling_rate)
    return audio

In [102]:
import os
audio_path = r'C:\Users\AlxMa\Desktop\Maestría\1er semestre\Sistemas reconfigurables\miedo.wav'
audio = load_audio(audio_path)

In [106]:
def extract_features(data):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T,axis=0)
    result = np.hstack((result,zcr)) #stacking horizontally

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sampling_rate).T,axis=0)
    resutl = np.hstack((result,chroma_stft))

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, mfcc)) 

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms)) 

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sampling_rate).T, axis=0)
    result = np.hstack((result, mel)) 
    
    print(result.shape)
    return result

In [107]:
features = extract_features(audio)
features.shape
features = features.reshape(1,-1)
prediction = model.predict(features)

(150,)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step


In [108]:
print(prediction[0])

pred_int = [(max(enumerate(prediction[0]), key=lambda x: x[1])[0])]
print(pred_int)
# Hacer conversión a valores enteros
label_map = {0:'anger', 1:'disgust', 2:'fear', 3:'happiness', 4:'neutral', 
             5:'sadness'}

pred_label = [label_map[l] for l in pred_int]
print(pred_label)

[7.5775208e-03 1.0552799e-02 1.3656817e-04 9.2233032e-01 6.4663362e-04
 5.8756176e-02]
[3]
['happiness']
